In [0]:
import os
import pandas as pd
from fastai.vision import *
from fastai.metrics import error_rate
import numpy as np

In [0]:
# create validation set
img_list = pd.read_csv(os.getcwd()+'/data/driver_imgs_list.csv')
valid_subjects = img_list.subject.sort_values().unique()[-6:]
img_list['is_valid'] = img_list['subject'].isin(valid_subjects)
print("valid subjects: ", valid_subjects)
print(img_list[img_list['is_valid']==True].subject.count())
img_list['img_path'] = img_list.classname + '/' + img_list.img
valid_names = img_list[img_list['subject'].isin(valid_subjects)].img
valid_names = valid_names.to_list()

In [0]:
# data loading
data = (ImageList.from_df(df=img_list, path = 'data/imgs/train/', cols='img_path')
        .split_by_valid_func(lambda o: os.path.basename(o) in valid_names)
        #.split_by_rand_pct(.2)
        .label_from_df(1)
        .transform(tfms=get_transforms(do_flip=False), size=224)
        .add_test_folder('../test')
        .databunch(bs=64))

In [0]:
from fastai.torch_core import *

import torch.nn as nn
import torch,math,sys
import torch.utils.model_zoo as model_zoo
from functools import partial

__all__ = ['XResNet', 'xresnet18', 'xresnet34', 'xresnet50', 'xresnet101', 'xresnet152']

In [0]:
from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastprogress import fastprogress
from torchvision.models import *
from functools import partial

In [0]:
act_fn = nn.ReLU(inplace=True)
def conv1d(ni:int, no:int, ks:int=1, stride:int=1, padding:int=0, bias:bool=False):
  "Create and initialize a `nn.Conv1d` layer with spectral normalization."
  conv = nn.Conv1d(ni, no, ks, stride=stride, padding=padding, bias=bias)
  nn.init.kaiming_normal_(conv.weight)
  if bias: conv.bias.data.zero_()
  return spectral_norm(conv)

class SimpleSelfAttention(nn.Module):
  def __init__(self, n_in:int, ks=1, sym=False):#, n_out:int):
    super().__init__()      
    self.conv = conv1d(n_in, n_in, ks, padding=ks//2, bias=False)      
    self.gamma = nn.Parameter(tensor([0.]))    
    self.sym = sym
    self.n_in = n_in

  def forward(self,x):      
    if self.sym:
      # symmetry hack by https://github.com/mgrankin
      c = self.conv.weight.view(self.n_in,self.n_in)
      c = (c + c.t())/2
      self.conv.weight = c.view(self.n_in,self.n_in,1)

    size = x.size()  
    x = x.view(*size[:2],-1)   # (C,N)
      
    # changed the order of mutiplication to avoid O(N^2) complexity
    # (x*xT)*(W*x) instead of (x*(xT*(W*x)))
    convx = self.conv(x)   # (C,C) * (C,N) = (C,N)   => O(NC^2)
    xxT = torch.bmm(x,x.permute(0,2,1).contiguous())   # (C,N) * (N,C) = (C,C)   => O(NC^2)
    o = torch.bmm(xxT, convx)   # (C,C) * (C,N) = (C,N)   => O(NC^2)
    o = self.gamma * o + x
    return o.view(*size).contiguous()

In [0]:
class Flatten(nn.Module):
    def forward(self, x): return x.view(x.size(0), -1)

def init_cnn(m):
    if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
    if isinstance(m, (nn.Conv2d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
    for l in m.children(): init_cnn(l)

def conv(ni, nf, ks=3, stride=1, bias=False):
    return nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=bias)

def noop(x): return x

def conv_layer(ni, nf, ks=3, stride=1, zero_bn=False, act=True):
    bn = nn.BatchNorm2d(nf)
    nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
    layers = [conv(ni, nf, ks, stride=stride), bn]
    if act: layers.append(act_fn)
    return nn.Sequential(*layers)

class ResBlock(nn.Module):
    def __init__(self, expansion, ni, nh, stride=1,sa=False, sym=False):
        super().__init__()
        nf,ni = nh*expansion,ni*expansion
        layers  = [conv_layer(ni, nh, 3, stride=stride),
                   conv_layer(nh, nf, 3, zero_bn=True, act=False)
        ] if expansion == 1 else [
                   conv_layer(ni, nh, 1),
                   conv_layer(nh, nh, 3, stride=stride),
                   conv_layer(nh, nf, 1, zero_bn=True, act=False)
        ]
        
        self.sa = SimpleSelfAttention(nf,ks=1,sym=sym) if sa else noop
        
        self.convs = nn.Sequential(*layers)
        # TODO: check whether act=True works better
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
        self.pool = noop if stride==1 else nn.AvgPool2d(2, ceil_mode=True)

    def forward(self, x): return act_fn(self.sa(self.convs(x)) + self.idconv(self.pool(x)))

def filt_sz(recep): return min(64, 2**math.floor(math.log2(recep*0.75)))
    
class XResNet(nn.Sequential):
    def __init__(self, expansion, layers, c_in=3, c_out=1000, sa = False, sym= False):
        
        
        stem = []
        sizes = [c_in,32,64,64]
        for i in range(3):
            stem.append(conv_layer(sizes[i], sizes[i+1], stride=2 if i==0 else 1))
            #nf = filt_sz(c_in*9)
            #stem.append(conv_layer(c_in, nf, stride=2 if i==1 else 1))
            #c_in = nf

        block_szs = [64//expansion,64,128,256,512]
        blocks = [self._make_layer(expansion, block_szs[i], block_szs[i+1], l, 1 if i==0 else 2, sa = sa if i in[len(layers)-2] else False, sym=sym)
                  for i,l in enumerate(layers)]
        super().__init__(
            *stem,
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            *blocks,
            nn.AdaptiveAvgPool2d(1), 
            Flatten(),
            nn.BatchNorm1d(block_szs[-1]*expansion, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            #nn.Dropout(p=0.15, inplace=False),
            #nn.Linear(block_szs[-1]*expansion, int(block_szs[-1]*expansion/4)),
            nn.Linear(block_szs[-1]*expansion, c_out, bias=True),
            #nn.ReLU(inplace=True),
            #nn.BatchNorm1d(int(block_szs[-1]*expansion/4), eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            #nn.Dropout(p=0.5, inplace=False),
            #nn.Linear(in_features=int(block_szs[-1]*expansion/4), out_features=c_out,bias=True),
            #nn.Softmax(),
        )
        init_cnn(self)

    def _make_layer(self, expansion, ni, nf, blocks, stride, sa=False, sym=False):
        return nn.Sequential(
            *[ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1, sa if i in [blocks -1] else False,sym)
              for i in range(blocks)])

def xresnet(expansion, n_layers, name, pretrained=False,  **kwargs):
    model = XResNet(expansion, n_layers, **kwargs)
    if pretrained: 
        model.load_state_dict(model_zoo.load_url('https://download.pytorch.org/models/resnet50-19c8e357.pth'))
    return model

me = sys.modules[__name__]
for n,e,l in [
    [ 18 , 1, [2,2,2 ,2] ],
    [ 34 , 1, [3,4,6 ,3] ],
    [ 50 , 4, [3,4,6 ,3] ],
    [ 101, 4, [3,4,23,3] ],
    [ 152, 4, [3,8,36,3] ],
]:
    name = f'xresnet{n}'
    setattr(me, name, partial(xresnet, expansion=e, n_layers=l, name=name))

In [0]:
torch.backends.cudnn.benchmark = True
fastprogress.MAX_COLS = 80
self_attention = 1
symmetry = 0
mom = 0.9 # parameter momentum
alpha = 0.99 # parameter alpha
eps = 1e-6 # parameter epsilon
opt = 'adam' # 'rms', 'sgd'
if opt=='adam': opt_func = partial(optim.Adam, betas=(mom,alpha), eps=eps)
elif opt=='rms': opt_func = partial(optim.RMSprop, alpha=alpha, eps=eps)
elif opt=='sgd': opt_func = partial(optim.SGD, momentum=mom)
#log_cb = partial(CSVLogger,filename=log)
m = globals()['xresnet50']

In [0]:
learn = (Learner(data, m(c_out=10, sa=self_attention, sym=symmetry), wd=1e-2, opt_func=opt_func,
             metrics=[accuracy, error_rate, FBeta(average='macro')],
             bn_wd=False, true_wd=True,
             loss_func = LabelSmoothingCrossEntropy()
             ))

In [0]:
lr = 1e-3 # learning rate
epochs = 5
learn = learn.to_fp16(dynamic=True)
learn.unfreeze()
learn.fit_one_cycle(epochs, max_lr=lr)

In [0]:
# The best result we have for the validation set.
learn.fit_one_cycle(1, max_lr=1e-8)

epoch,train_loss,valid_loss,accuracy,error_rate,f_beta,time
0,0.504300,1.012736,0.803917,0.196083,0.805633,02:33
